<a href="https://colab.research.google.com/github/ToiznerD/CloudComputing/blob/main/Project/user_interface_dor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install firebase
!pip install firebase-admin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [55]:
from zmq.constants import THREAD_AFFINITY_CPU_REMOVE
import json
from firebase import firebase
import IPython
from IPython.display import HTML
from google.colab import output, drive
from IPython.display import clear_output
from IPython.display import Markdown
import time
import ipywidgets as widgets
from google.colab.output import _publish as publish
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore, db

#cred = credentials.Certificate("/content/drive/My Drive/MyStyles/adminsdk.json")
#app = firebase_admin.initialize_app(cred)

def logo():
  image_widget = widgets.Image(value=open('/content/drive/My Drive/MyStyles/logo.jpeg', 'rb').read(), format='jpeg')
  image_widget.layout.width = '400px'
  image_widget.layout.height = 'auto'
  imgContainer = widgets.VBox([image_widget], layout=widgets.Layout(align_items='center'))
  display(imgContainer)


def userNav():
  display(HTML(f'<div class="login">Hello {user["username"]}! Total Points: {user["points"]}</div>'))

# Get style
drive.mount('/content/drive', force_remount=True)
with open('/content/drive/My Drive/MyStyles/style.css') as fid:
  style = fid.read()


questions = []
rAnswers=0
wAnswers=0
qnum = 0
qnumLabel = None
questionLabel = None
container = None
user = {}
################# LOGIN
def loginButton_eventhandler(obj):
  global user
  user = db.reference('Users/', app=firebase_admin.get_app(name="abc")).child(username.value).get()

  with open('/content/drive/My Drive/MyStyles/spinner.gif', 'rb') as f:
      img = f.read()
  #   create loading bar widget, ready to display when running long function
  loading_bar = widgets.Image(value=img, layout=widgets.Layout(width="25px", height="auto"))
  spinnerContainer = widgets.VBox([loading_bar], layout=widgets.Layout(width='100%', align_items='center'))
  display(spinnerContainer)
  time.sleep(7)
  spinnerContainer.children = []

  # Check if user is exists in DB
  if user:
      if user['password'] == password.value:
        label.value = 'You have successfully logged in!'

        time.sleep(2)
        loadGame()
      else:
        label.value = 'Wrong details or user is not exists'

publish.css(style)
logo()

# User input
userLabel = widgets.Label(value='User Name:', layout=widgets.Layout(width='100px'))
userLabel.add_class('login')
username = widgets.Text(layout=widgets.Layout(width='250px'))
userHbox = widgets.HBox([userLabel, username])

# Password input
pwLabel = widgets.Label(value='Password:', layout=widgets.Layout(width='100px'))
pwLabel.add_class('login')
password = widgets.Text(layout=widgets.Layout(width='250px'))
password.add_class('login')
pwHbox = widgets.HBox([pwLabel, password])

# Login button
btn = widgets.Button(description='Login')
btn.on_click(loginButton_eventhandler)
btn.add_class('button')

# Error label
label = widgets.Label(value='', layout=widgets.Layout(height='100px'))
label.add_class('login')

container = widgets.VBox([userHbox, pwHbox, btn, label], layout=widgets.Layout(width='100%', align_items='center'))
display(container)



################# LOAD GAME
def loadGame():
  clear_output(wait=True)
  ref = db.reference('Questions/', app=firebase_admin.get_app(name="abc")).get()
  for key, value in ref.items():
    if key != 'Count':
      questions.append(value)

  # Loading window

  publish.css(style)
  logo()
  userNav()
  display(IPython.display.HTML('''
  <html>
    <head>
      <h2>level 1</h2>
    </head>
    <body>

      <!-- Loading bar -->
      <div id="loading-bar">
        <div id="loading-progress"></div>
      </div>

      <!-- JavaScript code to simulate loading the game -->
      <script>
        var progress = 0;
        var loading = setInterval(function() {
          progress += Math.floor(Math.random() * 10); // simulate progress
          document.getElementById('loading-progress').style.width = progress + '%';
          if (progress >= 100) {
            clearInterval(loading); // stop the interval when progress reaches 100%
          }
        }, 500);
      </script>
    </body>
  </html>
  '''))

  time.sleep(2)

  # Clear loading window
  startGame()


################# START GAME
def startGame():
  clear_output(wait=True)
  ## START GAME UTILS ##
  def reloadGame():
    global wAnswers, rAnswers, qnum, questions
    wAnswers = 0
    rAnswers = 0
    qnum = 0
    questions = []
    loadGame()

  def checkAnswer(ans, buttons):
    cButton = None
    global qnum

    # Check if container is visible
    if container.layout.display == 'none':
      return

    # Find the correct answer
    for b in buttons:
      if b.description == questions[qnum]['correct']:
        cButton = b
    
    # Validate the clicked answer
    if ans.description == cButton.description:
      ans.add_class('correctAnswer')
      global rAnswers
      rAnswers += 1
    else:
      ans.add_class('wrongAnswer')
      cButton.add_class('correctAnswer')
      global wAnswers
      wAnswers += 1
    
    time.sleep(3)
    
    qnum += 1
    if qnum == len(questions): # in case of last question
      container.layout.display = 'none'
      rAnswersLabel = widgets.HTML(value=f'Correct Answers: {rAnswers}')
      rAnswersLabel.add_class('h5')
      wAnswersLabel = widgets.HTML(value=f'Wrong Answers: {wAnswers}')
      wAnswersLabel.add_class('h4')
      bonusPtsLabel = widgets.HTML(value=f'You get {rAnswers*10} new points!')
      bonusPtsLabel.add_class('h6')
      playAgain = widgets.Button(description="Play Again!", layout={'width': '350px','height': '100px', 'margin': '10px'})
      playAgain.add_class("button")
      playAgain.on_click(lambda button: reloadGame())
      answersContainer = widgets.VBox([rAnswersLabel, wAnswersLabel, bonusPtsLabel, playAgain], layout=widgets.Layout(align_items='center', height="350px"))
      display(answersContainer)

      user['points'] += rAnswers*10
      db.reference('Users/', app=firebase_admin.get_app(name="abc")).child(user['username']).update(user)
      return
    
    # Present next question
    questionLabel.value = questions[qnum]['question']
    qnumLabel.value = f'Question #{qnum+1}'
    for i,b in enumerate(buttons):
      ansStr = ansStr = f'ans{i+1}'
      b.description = questions[qnum][ansStr]
      b.remove_class("wrongAnswer")
      b.remove_class("correctAnswer")
      b.add_class("button")
  
  ## END UTILS ##

  publish.css(style)

  # Present logo
  logo()
  userNav()

  button_layout = {'width': '350px','height': '100px', 'margin': '10px'}
  buttons = []
  for i in range(1,5):
    ansStr = f'ans{i}'
    b = widgets.Button(description=questions[qnum][ansStr], layout=button_layout)
    b.add_class("button")
    buttons.append(b)

  button_container = widgets.HBox(children=buttons)

  # Add listeners
  for b in buttons:
    b.on_click(lambda button: checkAnswer(button, buttons))


  # Wrap the button container in a VBox container and center it horizontally
  qnumLabel = widgets.HTML(value=f'Question #{qnum+1}')
  qnumLabel.add_class("h2")
  questionLabel = widgets.HTML(value=questions[0]['question'])
  questionLabel.add_class("h2")

  container = widgets.VBox([qnumLabel, questionLabel, button_container], layout=widgets.Layout(align_items='center'))
  display(container)


In [43]:

import firebase_admin
from firebase_admin import credentials, db

# Initialize the Firebase Admin SDK
cred = credentials.Certificate('/content/drive/My Drive/MyStyles/adminsdk.json')
firebase_admin.initialize_app(cred,{'databaseURL': 'https://cloudcomputing-e1f5d-default-rtdb.firebaseio.com/'}, name="abc")


print(db.reference('Users/', app=firebase_admin.get_app(name="abc")).get())


{'amit': {'password': '123', 'points': 290, 'username': 'amit'}, 'benshi': {'password': '123', 'points': 390, 'username': 'benshi'}, 'dor': {'password': '123', 'points': 580, 'username': 'dor'}, 'raz': {'password': '123', 'points': 460, 'username': 'raz'}}
